In [1]:
import torchio as tio

In [2]:
img = tio.ScalarImage('/home/johannes/Data/SSD_2.0TB/GNN_pCR/data/brain_cancer/Lumiere/processed/Patient-001/week-000-1/CT1_r2s_bet_reg.nii.gz')

In [6]:
img_aligned = tio.ToCanonical()(img)

In [ ]:
img_resampled = tio.Resample(target=(0.5, 0.5, 0.75))

ScalarImage(shape: (1, 230, 230, 144); spacing: (1.00, 1.00, 1.00); orientation: RAS+; dtype: torch.FloatTensor; memory: 29.1 MiB)

In [6]:
import torch
import pandas as pd

In [4]:
subjects = torch.load("/home/johannes/Data/SSD_2.0TB/GNN_pCR/data/breast_cancer/final_patient_ids_4_timepoints.pt")
len(subjects)

585

In [9]:
df = pd.read_excel("/home/johannes/Data/SSD_2.0TB/GNN_pCR/data/breast_cancer/ISPY2-Imaging-Cohort-1-Clinical-Data.xlsx")
df

,Patient_ID,Arm,HR,HER2,MP,pCR,Age_at_Screening,Race,menopausal_status,ethnicity
0,756412,Paclitaxel + ABT 888 + Carboplatin,1,0,0,0,46.0,White,Premenopausal(< 6 months since LMP AND no prio...,Not Hispanic or Latino
1,111881,Paclitaxel + Neratinib,1,1,1,1,37.0,White,Premenopausal(<6 months since LMP AND no prior...,Not Hispanic or Latino
2,451816,Paclitaxel + ABT 888 + Carboplatin,1,0,0,0,43.0,White,Premenopausal(< 6 months since LMP AND no prio...,Not Hispanic or Latino
3,243836,Paclitaxel + Neratinib,1,0,1,1,31.0,White,Premenopausal(< 6 months since LMP AND no prio...,Not Hispanic or Latino
4,402265,Paclitaxel + Trastuzumab,1,1,0,0,56.0,White,Postmenopausal (prior bilateral ovariectomy OR...,Not Hispanic or Latino
...,...,...,...,...,...,...,...,...,...,...
980,231633,Paclitaxel + Pembrolizumab,0,0,1,1,53.0,White,Postmenopausal (prior bilateral ovariectomy OR...,Not Hispanic or Latino
981,892390,Paclitaxel + Pembrolizumab,0,0,1,1,59.0,White,Postmenopausal (prior bilateral ovariectomy OR...,Not Hispanic or Latino
982,666715,Paclitaxel + Ganetespib,1,0,0,0,32.0,White,Premenopausal(<6 months since LMP AND no prior...,Hispanic or Latino
983,241998,Paclitaxel,1,0,0,0,54.0,Black or African American,Postmenopausal (prior bilateral ovariectomy OR...,Not Hispanic or Latino


In [29]:
labels = []

for subject in subjects:
    
    subject_id = int(subject.split("-")[1])

    try:
    
        pcr_label = df[df["Patient_ID"] == subject_id]["pCR"].item()
        labels.append(pcr_label)
    except:
        pass

In [34]:
df = pd.DataFrame({"subject": subjects, "pCR": labels})
df.pCR.value_counts()

pCR
0    381
1    204
Name: count, dtype: int64